In [1]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from network import CharacterClassifier
from tqdm import tqdm
from training_data import HASY
from PIL import Image
from matplotlib import pyplot as plt

hasy_train = HASY('HASY')
hasy_test = HASY('HASY', train=False)

print("Train data length: {0}".format(len(hasy_train.data)))
print("Test data length: {0}".format(len(hasy_test.data)))
print("Img Shape: {0}".format(hasy_train.data[0].shape))
print("Number of Labels: {0}".format(hasy_train.no_labels))

100%|██████████| 16992/16992 [00:00<00:00, 52521.61it/s]

Train data length: 6550
Test data length: 372
Img Shape: torch.Size([3, 32, 32])
Number of Labels: 19


In [7]:
from training_data import OwnImgs
from torchvision import models

# Training Iteration with our images
hasy_model = models.densenet201(num_classes=19)
own_imgs = OwnImgs()

print("Train data length: {0}".format(len(own_imgs.data)))
print("Img Shape: {0}".format(own_imgs.data[0].shape))
print("Number of Labels: {0}".format(own_imgs.no_labels))

img_loader = DataLoader(own_imgs, batch_size=16, shuffle=True)
hasy_model.load_state_dict(torch.load('pretrained-model.ckpt'))
test_loader = DataLoader(hasy_test, batch_size=16, shuffle=False)

# Small accuracy test
accuracies = []
for idx, [x_test, y_test] in enumerate(tqdm(test_loader, desc='Test')):
    test_pred = hasy_model(x_test)
    accuracy = 100 * torch.mean((torch.argmax(test_pred, dim=1) == y_test).float())
    accuracies.append(accuracy)
print("Accuracy: {0}".format(np.mean(accuracies)))  

epochs = 15
optimizer = torch.optim.Adam(hasy_model.parameters(), lr=0.001, weight_decay=0.001)
criterion = nn.CrossEntropyLoss()
for epoch in range(epochs):
    print("Epoch {0}".format(epoch))
    for step, [x_train, y_train] in enumerate(tqdm(img_loader)):
        optimizer.zero_grad()
        train_pred = hasy_model(x_train)
        loss = criterion(train_pred, y_train)
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            print('Loss: {}'.format(loss))
torch.save(hasy_model.state_dict(), 'hasy_model-trans.ckpt')  

No training data for z. Skipping
Train data length: 1472
Img Shape: torch.Size([3, 32, 32])
Number of Labels: 19



  0%|          | 0/92 [00:00<?, ?it/s]

Accuracy: 33.33333206176758
Epoch 0



  1%|          | 1/92 [00:02<03:06,  2.05s/it]

Loss: 3.8471248149871826


...


  0%|          | 0/92 [00:00<?, ?it/s]

Epoch 14



  1%|          | 1/92 [00:02<03:21,  2.22s/it]

Loss: 0.5531014204025269



100%|██████████| 92/92 [03:43<00:00,  2.73s/it]